<a href="https://colab.research.google.com/github/altiphyl/Action-Redognition-i3d-UCF101/blob/main/ActionRecog_i3d_UCF101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
# Setting up the notebook

!pip install -q imageio
!pip install -q opencv-python
!pip install -q git+https://github.com/tensorflow/docs
!pip install pytube

# TensorFlow and TF-Hub modules.
from absl import logging
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_docs.vis import embed

# modules to help with reading the UCF101 dataset.
import random
import re
import os
import tempfile
import ssl
import cv2
import numpy as np

# modules to display an animation using imageio.
import imageio
from IPython import display
from urllib import request  

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 56 kB 1.5 MB/s 


In [17]:
# Prepping the UCF101 dataset

# Utilities to fetch videos from UCF101 dataset
UCF_ROOT = "https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/"
_VIDEO_LIST = None
_CACHE_DIR = tempfile.mkdtemp()
unverified_context = ssl._create_unverified_context()

def list_ucf_videos():
  """Lists videos available in UCF101 dataset."""
  global _VIDEO_LIST
  if not _VIDEO_LIST:
    index = request.urlopen(UCF_ROOT, context=unverified_context).read().decode("utf-8")
    videos = re.findall("(v_[\w_]+\.avi)", index)
    _VIDEO_LIST = sorted(set(videos))
  return list(_VIDEO_LIST)

def fetch_ucf_video(video):
  """Fetchs a video and cache into local filesystem."""
  cache_path = os.path.join(_CACHE_DIR, video)
  if not os.path.exists(cache_path):
    urlpath = request.urljoin(UCF_ROOT, video)
    print("Fetching %s => %s" % (urlpath, cache_path))
    data = request.urlopen(urlpath, context=unverified_context).read()
    open(cache_path, "wb").write(data)
  return cache_path

In [18]:
# Helper functions

# Utilities to open video files using CV2
def crop_center_square(frame):
  y, x = frame.shape[0:2]
  min_dim = min(y, x)
  start_x = (x // 2) - (min_dim // 2)
  start_y = (y // 2) - (min_dim // 2)
  return frame[start_y:start_y+min_dim,start_x:start_x+min_dim]

def load_video(path, max_frames=0, resize=(224, 224)):
  cap = cv2.VideoCapture(path)
  frames = []
  try:
    while True:
      ret, frame = cap.read()
      if not ret:
        break
      frame = crop_center_square(frame)
      frame = cv2.resize(frame, resize)
      frame = frame[:, :, [2, 1, 0]]
      frames.append(frame)
      
      if len(frames) == max_frames:
        break
  finally:
    cap.release()
  return np.array(frames) / 255.0

def to_gif(images):
  converted_images = np.clip(images * 255, 0, 255).astype(np.uint8)
  imageio.mimsave('./animation.gif', converted_images, fps=25)
  return embed.embed_file('./animation.gif')

In [19]:
# Get the kinetics-400 action labels from the GitHub repository.
KINETICS_URL = "https://raw.githubusercontent.com/deepmind/kinetics-i3d/master/data/label_map.txt"
with request.urlopen(KINETICS_URL) as obj:
  labels = [line.decode("utf-8").strip() for line in obj.readlines()]
print("Found %d labels." % len(labels))

Found 400 labels.


In [47]:
# Get the list of videos in the dataset.
ucf_videos = list_ucf_videos()
  
categories = {}
for video in ucf_videos:
  category = video[2:-12]
  if category not in categories:
    categories[category] = []
  categories[category].append(video)
print("Found %d videos in %d categories." % (len(ucf_videos), len(categories)))


Found 13320 videos in 101 categories.


In [22]:
# Downloading the i3d model

i3d = hub.load("https://tfhub.dev/deepmind/i3d-kinetics-400/1").signatures['default']

In [13]:
def predict(sample_video):
  # Add a batch axis to the sample video.
  model_input = tf.constant(sample_video, dtype=tf.float32)[tf.newaxis, ...]

  logits = i3d(model_input)['default'][0]
  probabilities = tf.nn.softmax(logits)

  print("Most Probably Action:")
  for i in np.argsort(probabilities)[::-1][:1]:
    print(f"  {labels[i]:22}: {probabilities[i] * 100:5.2f}%")

In [24]:
# Testing:
video_path = fetch_ucf_video("v_Shotput_g01_c01.avi")
sample_video = load_video(video_path)

predict(sample_video)

Fetching https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/v_Shotput_g01_c01.avi => /tmp/tmpyweyj2g8/v_Shotput_g01_c01.avi
Most Probably Action:
  shot put              : 85.92%


PASSED!


Now we will test on random youtube videos :

In [31]:
import pytube

# TEST 1

link = "https://www.youtube.com/watch?v=JybiVxh1SWA"
yt = pytube.YouTube(link)
stream = yt.streams.filter(progressive=True, file_extension='mp4').first()
stream.download("/content/youtubevid")
video_path = "/content/youtubevid/Cartwheel clip.mp4"
sample_video = load_video(video_path)[:100]
to_gif(sample_video)
predict(sample_video)

Most Probably Action:
  gymnastics tumbling   : 40.07%


PASSED!


In [35]:
# TEST 2

link = "https://www.youtube.com/shorts/NUrZSB3dtSQ"
yt = pytube.YouTube(link)
stream = yt.streams.filter(progressive=True, file_extension='mp4').first()
stream.download("/content/youtubevid")
video_path = "/content/youtubevid/The different types of runners.mp4"
sample_video = load_video(video_path)[:100]
to_gif(sample_video)
predict(sample_video)

Most Probably Action:
  jogging               : 97.12%


PASSED!
